In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'

import torch
from fastai.vision import *
import superRes.fid_score as fs
from superRes.dataset import *
from superRes.fid_loss import *
from superRes.inception import InceptionV3

In [2]:
path = untar_data(URLs.PETS)

path_fullRes = path/'images'
path_sampleImagesFullRes = path/'sampleImagesFullRes' # 10 img sample full res
path_sampleImagesLowRes = path/'sampleImagesLowRes'# same 10 img sample but low res

low = ImageList.from_folder(path_sampleImagesLowRes)
full = ImageList.from_folder(path_sampleImagesFullRes)

In [3]:
def get_databunch(
    sz: int,
    bs: int,
    crappy_path: Path,
    good_path: Path,
    random_seed: int = 42,
    keep_pct: float = 1.0,
    num_workers: int = 8,
    xtra_tfms=[],
) -> ImageDataBunch:

    src = (
        ImageImageList.from_folder(crappy_path)
        .split_none()
    )

    data = (
        src.label_from_func(lambda x: good_path / x.relative_to(crappy_path))
        .transform(
            get_transforms(
                max_zoom=1.
            ),
            size=sz,
            tfm_y=True,
        )
        .databunch(bs=bs, num_workers=num_workers, no_check=True)
    )

    data.c = 3
    return data

def get_data(bs:int, sz:int, keep_pct:float):
    return get_databunch(sz=sz, bs=bs, crappy_path=path_sampleImagesLowRes, 
                         good_path=path_sampleImagesFullRes, 
                         random_seed=None, keep_pct=keep_pct)

In [4]:
data = get_data(bs=10, sz=256, keep_pct=1.)
# data.show_batch()
x, y = data.one_batch()

In [5]:
calculate_fid(x, y, True, 2048)

37.65856072143356